- Nombre: Marcus Alexander Mayorga Martínez
- Fecha: 8 de mayo del 2025

Importar las librerías necesarias

In [35]:
from abc import ABC, abstractmethod

Clase abstracta Vehículo

In [36]:
class Vehiculo(ABC):
    """Clase abstracta base para todos los vehículos"""

    def __init__(self, identificador: str, modelo: str, velocidad_maxima: float, capacidad_carga: float):
        self._id = identificador
        self._modelo = modelo
        self._velocidad_maxima = velocidad_maxima
        self._capacidad_carga = capacidad_carga
        self._velocidad_actual = 0
        self._estrategia = None

    #Getters
    @property
    def modelo(self):
        return self._modelo

    @property
    def velocidad_maxima(self):
        return self._velocidad_maxima

    @property
    def velocidad_actual(self):
        return self._velocidad_actual

    #Métodos Base
    def set_estrategia(self, estrategia):
        self._estrategia = estrategia

    def ejecutar_estrategia(self):
        if self._estrategia:
            self._estrategia.conducir(self)

    #Métodos Abstractos
    @abstractmethod
    def acelerar(self):
        pass

    @abstractmethod
    def frenar(self):
        pass

    @abstractmethod
    def informar_estado(self):
        pass

    #Sobrecarga Operadores
    def __eq__(self, other):
        if not isinstance(other, Vehiculo):
            return False
        return (self.modelo == other.modelo and
                self.velocidad_maxima == other.velocidad_maxima)


SUBCLASES DE VEHÍCULOS

In [37]:
class Automovil(Vehiculo):
    def acelerar(self):
        self._velocidad_actual = min(self._velocidad_actual + 10, self._velocidad_maxima)

    def frenar(self):
        self._velocidad_actual = max(self._velocidad_actual - 15, 0)

    def informar_estado(self):
        print(f"Automóvil {self._id}: {self._velocidad_actual} km/h")

class Camion(Vehiculo):
    def __init__(self, *args):
        super().__init__(*args)
        self._remolques = 0

    def engranar_remolque(self):
        self._remolques += 1

    def acelerar(self):
        self._velocidad_actual = min(self._velocidad_actual + 5, self._velocidad_maxima)

    def frenar(self):
        self._velocidad_actual = max(self._velocidad_actual - 20, 0)

    def informar_estado(self):
        print(f"Camión {self._id}: {self._remolques} remolques | {self._velocidad_actual} km/h")

class Motocicleta(Vehiculo):
    def acelerar(self):
        self._velocidad_actual = min(self._velocidad_actual + 15, self._velocidad_maxima)

    def frenar(self):
        self._velocidad_actual = max(self._velocidad_actual - 10, 0)

    def realizar_maniobra_evasiva(self):
        print(f"Motocicleta {self._id} realiza maniobra evasiva!")
        self._velocidad_actual = max(self._velocidad_actual - 30, 0)

    def informar_estado(self):
        print(f"Motocicleta {self._id}: {self._velocidad_actual} km/h")

STRATEGY PATTERN

In [38]:
class EstrategiaConduccion(ABC):
    @abstractmethod
    def conducir(self, vehiculo: Vehiculo):
        pass

class ConduccionEconomica(EstrategiaConduccion):
    def conducir(self, vehiculo):
        vehiculo.acelerar()
        if vehiculo.velocidad_actual > 60:
            vehiculo.frenar()

class ConduccionDeportiva(EstrategiaConduccion):
    def conducir(self, vehiculo):
        vehiculo.acelerar()
        vehiculo.acelerar()


# Conducción de emergencia (Reto adicional)
class ConduccionEmergencia(EstrategiaConduccion):
    def conducir(self, vehiculo):
        # Permitir exceder velocidad máxima en un 20% solo durante emergencia
        limite_emergencia = vehiculo.velocidad_maxima * 1.2
        incremento = 30  # Acelera rápidamente
        nueva_velocidad = vehiculo._velocidad_actual + incremento
        vehiculo._velocidad_actual = min(nueva_velocidad, limite_emergencia)
        print(f"[EMERGENCIA] {vehiculo.modelo} alcanza {vehiculo._velocidad_actual:.1f} km/h")

DECORATOR PATTERN

In [39]:
class VehiculoDecorator(Vehiculo):
    def __init__(self, vehiculo: Vehiculo):
        self._vehiculo = vehiculo

    @property
    def modelo(self):
        return self._vehiculo.modelo

    @property
    def velocidad_maxima(self):
        return self._vehiculo.velocidad_maxima

    def acelerar(self):
        self._vehiculo.acelerar()

    def frenar(self):
        self._vehiculo.frenar()

    def informar_estado(self):
        self._vehiculo.informar_estado()

class PilotoAutomaticoDecorator(VehiculoDecorator):
    def informar_estado(self):
        print("[Piloto Automático] ", end="")
        super().informar_estado()

class AsistenteEstacionamientoDecorator(VehiculoDecorator):
    def estacionar(self):
        self._vehiculo.frenar()
        print(f"Vehículo {self._vehiculo._id} estacionado automáticamente")

SINGLETON

In [40]:
class ControlDeFlota:
    _instancia = None

    def __new__(cls):
        if not cls._instancia:
            cls._instancia = super().__new__(cls)
            cls._instancia.vehiculos = []
        return cls._instancia

    def __add__(self, vehiculo):
        self.vehiculos.append(vehiculo)
        return self

    def mostrar_flota(self):
        print("\n--- Vehículos en flota ---")
        for vehiculo in self.vehiculos:
            vehiculo.informar_estado()

Código de prueba

In [41]:
if __name__ == "__main__":
    # Creación de vehículos
    auto_base = Automovil("A-001", "Tesla Model S", 250, 500)
    camion_base = Camion("C-001", "Volvo FH16", 120, 5000)
    moto_base = Motocicleta("M-001", "Harley Davidson", 180, 150)

    # Aplicación de decoradores
    auto_decorado = PilotoAutomaticoDecorator(auto_base)
    camion_decorado = AsistenteEstacionamientoDecorator(camion_base)

    # Configuración de estrategias
    auto_base.set_estrategia(ConduccionDeportiva())
    camion_base.set_estrategia(ConduccionEconomica())
    moto_base.set_estrategia(ConduccionEconomica())

    # Configuración de flota
    flota = ControlDeFlota()
    flota + auto_decorado + camion_decorado + moto_base

    # Simulación de conducción
    print("\n--- Primera iteración de estrategias ---")
    auto_base.ejecutar_estrategia()
    camion_base.ejecutar_estrategia()
    moto_base.ejecutar_estrategia()

    print("\n--- Segunda iteración de estrategias ---")
    auto_base.ejecutar_estrategia()
    camion_base.ejecutar_estrategia()

    # Maniobras especiales
    print("\n--- Maniobras especiales ---")
    camion_base.engranar_remolque()
    moto_base.realizar_maniobra_evasiva()

    # Uso de asistente estacionamiento
    print("\n--- Asistente estacionamiento ---")
    camion_decorado.estacionar()

    # Mostrar estado final
    print("\n--- Estado final de la flota ---")
    flota.mostrar_flota()

    # Prueba de igualdad
    otro_auto = Automovil("A-002", "Tesla Model S", 250, 500)
    print(f"\n¿Autos iguales? {auto_base == otro_auto}")
    print(f"¿Auto igual a camión? {auto_base == camion_base}")

    # Prueba Singleton
    print("\n--- Prueba Singleton ---")
    otra_flota = ControlDeFlota()
    otra_flota.mostrar_flota()



    # Imprimir Conducción de emergencia (Reto adicional)
    print("\n--- Conducción de emergencia ---")
    auto_base.set_estrategia(ConduccionEmergencia())
    auto_base.ejecutar_estrategia()



--- Primera iteración de estrategias ---

--- Segunda iteración de estrategias ---

--- Maniobras especiales ---
Motocicleta M-001 realiza maniobra evasiva!

--- Asistente estacionamiento ---
Vehículo C-001 estacionado automáticamente

--- Estado final de la flota ---

--- Vehículos en flota ---
[Piloto Automático] Automóvil A-001: 40 km/h
Camión C-001: 1 remolques | 0 km/h
Motocicleta M-001: 0 km/h

¿Autos iguales? True
¿Auto igual a camión? False

--- Prueba Singleton ---

--- Vehículos en flota ---
[Piloto Automático] Automóvil A-001: 40 km/h
Camión C-001: 1 remolques | 0 km/h
Motocicleta M-001: 0 km/h

--- Conducción de emergencia ---
[EMERGENCIA] Tesla Model S alcanza 70.0 km/h
